In [5]:
import time
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import json
from lib.mc_net import mc_model, costCriterionReaching
from lib.some import reduct
import seaborn as sns

In [6]:
# time-settings
dh = 1e-2 # 10 ms sim-step
duration = 1.2 # 600+200 ms
T = int(round(duration/dh))
# load inputs
with open('TARGET_train.json') as f:
    prms = json.load(f)    
home_joint_state = np.array(prms['home_join'])
home_cart_state = np.array(prms['home_cart'])
cart_targ = np.array(prms['cart_targ'])
cart_targ = torch.from_numpy(cart_targ).float()
vel = np.array(prms['nor_vel'])
vel = torch.from_numpy(vel[:-10]).float()
# colormap
cm = sns.color_palette("RdYlBu", 8)
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE' # you can neglect this (just my env met some problems)

# baseline model 

In [ ]:
os.makedirs('isn_o', exist_ok=True) 
os.makedirs('log', exist_ok=True) 
seed = 2023
torch.manual_seed(seed)            
torch.cuda.manual_seed(seed) 
reach_sim = mc_model(dh, torch.from_numpy(home_joint_state).float(), torch.from_numpy(home_cart_state).float())

lrate = 2e-4 

optim_params = (
    [reach_sim.xstars_prms] +
    list(reach_sim.mc_inplayer.parameters()) +
    [reach_sim.c_prms]
)

optimizer = optim.Adam(optim_params, lr=lrate, weight_decay=1e-6)   
# scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9991) # lr_now = lr * exp(step of epoch)
# optim.lr_scheduler.StepLR

# inters
inters = 1000
total_loss_for_plotting = np.empty(0)

tic = time.time()
for it in range(inters):

    optimizer.zero_grad() 
    
    kinematics = reach_sim.forward(T, cart_targ, 'all')
    loss = costCriterionReaching(reach_sim, cart_targ, torch.stack(kinematics))  
    loss.backward(retain_graph=True) 
    optimizer.step()

    # learning rate schedual
    # scheduler.step() 
    total_loss_for_plotting = np.append(total_loss_for_plotting, loss.item())

    sys.stdout.write(f'\r iteration {it+1}/{inters} | train loss: {loss.item():.5f}')
    sys.stdout.flush()
    plt.plot(total_loss_for_plotting)
    plt.title('The loss is %.5f of iter %d'%(loss.item(),it))

    plt.savefig('log/isn.png')
    plt.clf()  # release the memory of figure
    plt.cla()  
    if (it+1)%100 == 0 and it>500:
        fig, axs = plt.subplots(2, 3, figsize=(7*3, 5*2)) 
        axs = axs.ravel()
        axs[0].plot(torch.stack(kinematics)[:,:,0].data.numpy(), torch.stack(kinematics)[:,:,1].data.numpy())        
        axs[0].plot(cart_targ[:,0].data.numpy(), cart_targ[:,1].data.numpy(), 'o')
        axs[0] .set_title('joint space')
        
        axs[1].plot(torch.stack(reach_sim.inp_list)[:, 0, :].data.numpy())
        axs[1] .set_title('network input before tanh')
        
        axs[2].plot(torch.stack(reach_sim.networkactivity_list)[:, 0, :200].data.numpy())
        axs[2] .set_title('neural activity')
        
        axs[3].plot(torch.stack(reach_sim.mus_out_list)[:, 0, :].data.numpy())
        axs[3] .set_title('muscle activity')
        
        axs[4].plot(torch.stack(kinematics)[:,:,2].data.numpy())  
        axs[4] .set_title('velocity')
        
        axs[5].plot(torch.stack(reach_sim.networkactivity_list)[:, -2, :200].data.numpy())
        axs[5] .set_title('neural activity')
        plt.savefig("isn_o/m_ep{}_{}.png".format(it+1,total_loss_for_plotting[it]))
        plt.close('all')
        torch.save(reach_sim,"isn_o/m_mep{}_{}.pt".format(it+1,total_loss_for_plotting[it]))

# no velocity feedback

In [4]:
os.makedirs('isn_o\\novel', exist_ok=True) 
seed = 2023
torch.manual_seed(seed)            
torch.cuda.manual_seed(seed) 
reach_sim = mc_model(dh, torch.from_numpy(home_joint_state).float(), torch.from_numpy(home_cart_state).float())

lrate = 2e-4 

optim_params = (
    [reach_sim.xstars_prms] +
    list(reach_sim.mc_inplayer.parameters()) +
    [reach_sim.c_prms]
)

optimizer = optim.Adam(optim_params, lr=lrate, weight_decay=1e-6)   
# scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9992) # lr_now = lr * exp(step of epoch)
# optim.lr_scheduler.StepLR

# inters
inters = 1000
total_loss_for_plotting_vel = np.empty(0)

tic = time.time()
for it in range(inters):

    optimizer.zero_grad() 
    
    kinematics = reach_sim.forward(T, cart_targ, 'novel')
    loss = costCriterionReaching(reach_sim, cart_targ, torch.stack(kinematics))     
    loss.backward(retain_graph=True) 
    optimizer.step()

    # learning rate schedual
    # scheduler.step() 
    total_loss_for_plotting_vel = np.append(total_loss_for_plotting_vel, loss.item())

    sys.stdout.write(f'\r iteration {it+1}/{inters} | train loss: {loss.item():.5f}')
    sys.stdout.flush()
    plt.plot(total_loss_for_plotting_vel)
    plt.title('The loss is %.5f of iter %d'%(loss.item(),it))

    plt.savefig('log/novel.png')
    plt.clf()  # release the memory of figure
    plt.cla()  
    if (it+1)%100 == 0 :
        fig, axs = plt.subplots(2, 3, figsize=(7*3, 5*2)) 
        axs = axs.ravel()
        axs[0].plot(torch.stack(kinematics)[:,:,0].data.numpy(), torch.stack(kinematics)[:,:,1].data.numpy())        
        axs[0].plot(cart_targ[:,0].data.numpy(), cart_targ[:,1].data.numpy(), 'o')
        axs[0] .set_title('joint space')
        
        axs[1].plot(torch.stack(reach_sim.inp_list)[:, 0, :].data.numpy())
        axs[1] .set_title('network input before tanh')
        
        axs[2].plot(torch.stack(reach_sim.networkactivity_list)[:, 0, :200].data.numpy())
        axs[2] .set_title('neural activity')
        
        axs[3].plot(torch.stack(reach_sim.mus_out_list)[:, 0, :].data.numpy())
        axs[3] .set_title('muscle activity')
        
        axs[4].plot(torch.stack(kinematics)[:,:,2].data.numpy())  
        axs[4] .set_title('velocity')
        
        axs[5].plot(torch.stack(reach_sim.networkactivity_list)[:, -2, :200].data.numpy())
        axs[5] .set_title('neural activity')
        plt.savefig("isn_o/novel/m_ep{}_{}.png".format(it+1,total_loss_for_plotting_vel[it]))

        plt.close('all')
        torch.save(reach_sim,"isn_o/novel/m_ep{}_{}.pt".format(it+1,total_loss_for_plotting_vel[it]))


 iteration 1000/1000 | train loss: 0.00388

# no muscle force feedback

In [5]:
os.makedirs('isn_o\\nomus', exist_ok=True) 
seed = 2023
torch.manual_seed(seed)            
torch.cuda.manual_seed(seed) 
reach_sim = mc_model(dh, torch.from_numpy(home_joint_state).float(), torch.from_numpy(home_cart_state).float())

lrate = 2e-4 
optim_params = (
    [reach_sim.xstars_prms] +
    list(reach_sim.mc_inplayer.parameters()) +
    [reach_sim.c_prms]
)

optimizer = optim.Adam(optim_params, lr=lrate, weight_decay=1e-6)   
# scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9992) # lr_now = lr * exp(step of epoch)
# optim.lr_scheduler.StepLR

# inters
inters = 1000

total_loss_for_plotting_mus = np.empty(0)

tic = time.time()
for it in range(inters):

    optimizer.zero_grad() 
    
    kinematics = reach_sim.forward(T, cart_targ, 'nomus')
    loss = costCriterionReaching(reach_sim, cart_targ, torch.stack(kinematics))    
    loss.backward(retain_graph=True) 
    optimizer.step()

    # learning rate schedual
    # scheduler.step() 
    total_loss_for_plotting_mus = np.append(total_loss_for_plotting_mus, loss.item())

    sys.stdout.write(f'\r iteration {it+1}/{inters} | train loss: {loss.item():.5f}')
    sys.stdout.flush()
    plt.plot(total_loss_for_plotting_mus)
    plt.title('The loss is %.5f of iter %d'%(loss.item(),it))

    plt.savefig('log/nmus.png')
    plt.clf()  # release the memory of figure
    plt.cla()  
    if (it+1)%100 == 0 :
        fig, axs = plt.subplots(2, 3, figsize=(7*3, 5*2)) 
        axs = axs.ravel()
        axs[0].plot(torch.stack(kinematics)[:,:,0].data.numpy(), torch.stack(kinematics)[:,:,1].data.numpy())        
        axs[0].plot(cart_targ[:,0].data.numpy(), cart_targ[:,1].data.numpy(), 'o')
        axs[0] .set_title('joint space')
        
        axs[1].plot(torch.stack(reach_sim.inp_list)[:, 0, :].data.numpy())
        axs[1] .set_title('network input before tanh')
        
        axs[2].plot(torch.stack(reach_sim.networkactivity_list)[:, 0, :200].data.numpy())
        axs[2] .set_title('neural activity')
        
        axs[3].plot(torch.stack(reach_sim.mus_out_list)[:, 0, :].data.numpy())
        axs[3] .set_title('muscle activity')
        
        axs[4].plot(torch.stack(kinematics)[:,:,2].data.numpy())  
        axs[4] .set_title('velocity')
        
        axs[5].plot(torch.stack(reach_sim.networkactivity_list)[:, -2, :200].data.numpy())
        axs[5] .set_title('neural activity')
        plt.savefig("isn_o/nomus/m_ep{}_{}.png".format(it+1,total_loss_for_plotting_mus[it]))

        plt.close('all')
        torch.save(reach_sim,"isn_o/nomus/m_ep{}_{}.pt".format(it+1,total_loss_for_plotting_mus[it]))


 iteration 1000/1000 | train loss: 0.00507

# no velocity and muscle force feedback

In [6]:
os.makedirs('isn_o\\only', exist_ok=True) 
seed = 2023
torch.manual_seed(seed)            
torch.cuda.manual_seed(seed) 
reach_sim = mc_model(dh, torch.from_numpy(home_joint_state).float(), torch.from_numpy(home_cart_state).float())

lrate = 2e-4 
optim_params = (
    [reach_sim.xstars_prms] +
    list(reach_sim.mc_inplayer.parameters()) +
    [reach_sim.c_prms]
)

optimizer = optim.Adam(optim_params, lr=lrate, weight_decay=1e-6)   
# scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9992) # lr_now = lr * exp(step of epoch)
# optim.lr_scheduler.StepLR

# inters
inters = 1000

total_loss_for_plotting_ol = np.empty(0)

tic = time.time()
for it in range(inters):

    optimizer.zero_grad() 
    
    kinematics = reach_sim.forward(T, cart_targ, 'only')
    loss = costCriterionReaching(reach_sim, cart_targ, torch.stack(kinematics))    
    loss.backward(retain_graph=True) 
    optimizer.step()

    # learning rate schedual
    # scheduler.step() 
    total_loss_for_plotting_ol = np.append(total_loss_for_plotting_ol, loss.item())

    sys.stdout.write(f'\r iteration {it+1}/{inters} | train loss: {loss.item():.5f}')
    sys.stdout.flush()
    plt.plot(total_loss_for_plotting_ol)
    plt.title('The loss is %.5f of iter %d'%(loss.item(),it))

    plt.savefig('log/only.png')
    plt.clf()  # release the memory of figure
    plt.cla()  
    if (it+1)%100 == 0 :
        fig, axs = plt.subplots(2, 3, figsize=(7*3, 5*2)) 
        axs = axs.ravel()
        axs[0].plot(torch.stack(kinematics)[:,:,0].data.numpy(), torch.stack(kinematics)[:,:,1].data.numpy())        
        axs[0].plot(cart_targ[:,0].data.numpy(), cart_targ[:,1].data.numpy(), 'o')
        axs[0] .set_title('joint space')
        
        axs[1].plot(torch.stack(reach_sim.inp_list)[:, 0, :].data.numpy())
        axs[1] .set_title('network input before tanh')
        
        axs[2].plot(torch.stack(reach_sim.networkactivity_list)[:, 0, :200].data.numpy())
        axs[2] .set_title('neural activity')
        
        axs[3].plot(torch.stack(reach_sim.mus_out_list)[:, 0, :].data.numpy())
        axs[3] .set_title('muscle activity')
        
        axs[4].plot(torch.stack(kinematics)[:,:,2].data.numpy())  
        axs[4] .set_title('velocity')
        
        axs[5].plot(torch.stack(reach_sim.networkactivity_list)[:, -2, :200].data.numpy())
        axs[5] .set_title('neural activity')
        plt.savefig("isn_o/only/m_ep{}_{}.png".format(it+1,total_loss_for_plotting_ol[it]))

        plt.close('all')
        torch.save(reach_sim,"isn_o/only/m_ep{}_{}.pt".format(it+1,total_loss_for_plotting_ol[it]))


 iteration 1000/1000 | train loss: 0.00408

# autonomous system

In [ ]:
from lib.auto import mc_model, costCriterionReaching

os.makedirs('isn_o\\auto', exist_ok=True) 

seed = 2023
torch.manual_seed(seed)            
torch.cuda.manual_seed(seed) 
reach_sim = mc_model(dh, torch.from_numpy(home_joint_state).float(), torch.from_numpy(home_cart_state).float())

lrate = 2e-4 
optim_params = (
    [reach_sim.xstars_prms] +
    list(reach_sim.mc_inplayer.parameters()) +
    [reach_sim.c_prms]
)

optimizer = optim.Adam(optim_params, lr=lrate, weight_decay=1e-6)  
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9991) 
# optim.lr_scheduler.StepLR

# inters
inters = 1000

total_loss_for_plotting = np.empty(0)

tic = time.time()
for it in range(inters):

    optimizer.zero_grad() 
    
    kinematics = reach_sim.forward(T, cart_targ)
    loss = costCriterionReaching(reach_sim, cart_targ, torch.stack(kinematics))    
    loss.backward(retain_graph=True) 
    optimizer.step()

    # learning rate schedual
    scheduler.step() 
    total_loss_for_plotting = np.append(total_loss_for_plotting, loss.item())

    sys.stdout.write(f'\r iteration {it+1}/{inters} | train loss: {loss.item():.5f}')
    sys.stdout.flush()
    plt.plot(total_loss_for_plotting)
    plt.title('The loss is %.5f of iter %d'%(loss.item(),it))
   
    # plt.savefig('log/chaos.png')
    plt.savefig('log/auto.png')
    plt.clf()  
    plt.cla()  
    if (it+1)%100 == 0 :
        fig, axs = plt.subplots(2, 3, figsize=(7*3, 5*2)) 
        axs = axs.ravel()
        axs[0].plot(torch.stack(kinematics)[:,:,0].data.numpy(), torch.stack(kinematics)[:,:,1].data.numpy())        
        axs[0].plot(cart_targ[:,0].data.numpy(), cart_targ[:,1].data.numpy(), 'o')
        axs[0] .set_title('joint space')
        
        # axs[1].plot(torch.stack(reach_sim.inp_list)[:, 0, :].data.numpy())
        # axs[1] .set_title('network input before tanh')
        
        axs[2].plot(torch.stack(reach_sim.networkactivity_list)[:, 0, :200].data.numpy())
        axs[2] .set_title('neural activity')
        
        axs[3].plot(torch.stack(reach_sim.mus_out_list)[:, 0, :].data.numpy())
        axs[3] .set_title('muscle activity')
        
        axs[4].plot(torch.stack(kinematics)[:,:,2].data.numpy())  
        axs[4] .set_title('velocity')

        axs[5].plot(torch.stack(reach_sim.networkactivity_list)[:, -2, :200].data.numpy())
        axs[5] .set_title('neural activity')
        plt.savefig("isn_o/auto/m_ep{}_{}.png".format(it+1,total_loss_for_plotting[it]))
        plt.close('all')
        torch.save(reach_sim,"isn_o/auto/m_ep{}_{}.pt".format(it+1,total_loss_for_plotting[it]))

# other combination of feedback see in the 'choice' in mc_net.py